In [1]:
from __future__ import print_function
import sys
from pyspark.streaming import StreamingContext
from pyspark import SparkContext,SparkConf
from pyspark.streaming.kafka import KafkaUtils
import json

import os
import sys
import ast
import json

import re
import string
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import threading
import time
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pylab
from pylab import rcParams
import numpy as np
import multiprocessing

from pyspark import SparkContext
from pyspark import SQLContext,Row
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.ml.feature import HashingTF,IDF, Tokenizer
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.feature import Word2VecModel

BATCH_INTERVAL = 15  # How frequently to update (seconds)
clusterNum=15
# Change the backend to TkAgg to plot - only for Mac users
#plt.switch_backend('TkAgg')

def get_coord(post):
    coord = tuple()
    g = geocoder.geonames(post,key="ragulaaryan")
    try:
        if len(post)==0:
            coord = (0,0)
        else:
            coord = (float(g.lat),float(g.lng))
    except TypeError:
        #print ('error get_coord')
        coord=(0,0)
    return coord

def get_cordinates_from_event(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree and subtree.label() == "GPE":
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        if current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue
            
    
    place=[]
    location=()
    if len(continuous_chunk)==0:
        location=(0,0)
        return location
    else:
        for item in range(len(continuous_chunk)):
            location=get_coord(continuous_chunk[item])
            return location
            break


def get_json_from_string(jsonString):
    '''
            Methos to convert the twitter feed event read from kafka to json
            Args:
                jsonString: even data as string
            Returns:
                json: converted json 
        '''
    
    try:
        json_object = json.loads(jsonString)
    except ValueError:
        return False
    return json_object

def media_url(tweet):
    if "media" in tweet['entities']:
        for image in tweet['entities']['media']:
            return image['media_url']
    try:
        if "media" in tweet['retweeted_status']['entities']:
            for image in  tweet['retweeted_status']['entities']['media']:
                return image['media_url']
    except KeyError:
        return False
    else:
        return False
    
def doc2vec(twitter_feed_document):
    '''
            Method to get the vectors for every feed text from word2vec model already built using Word2Vec spark mllib library
            Args:
                twitter_feed_document: tuple of feed words got after pre processing (remove stop words, stemming)
            Returns:
                vecor representation of words in the feed text
        '''
    doc_vector = np.zeros(100)
    total_words = 0

    for word in twitter_feed_document:
        try:
            vector = np.array(word2vec_model_brooadcast.value.get(word))
            if vec!= None:
                doc_vector +=  vector
                total_words += 1
        except:
            continue

    if total_words>0:
        return doc_vector / float(total_words)
    return doc_vector


# Reg ex to remove special characters from feed text
special_character_regex = re.compile('[%s]' % re.escape(string.punctuation)) 
# list of stop words to remove, extracted from nltk python library 
stopwords=[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now']


def tokenize(text):
    '''
            Method to remove special characters, stop words from text and tokenize 
            Args:
                text: actual text to be tokenized
            Returns:
                list of tokens generated
        '''
    tokens = []
    #text = text.encode('ascii', 'ignore') #to decode
    text=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text) # to replace url with ''
    text = special_character_regex.sub(" ",text)  # Remove special characters
    text=text.lower()

    for word in text.split():
        if word not in stopwords \
            and word not in string.punctuation \
            and len(word)>1 \
            and word != '``':
                tokens.append(word)
    return tokens

from collections import Counter
def get_most_popular_words(terms):
    '''
            Method to get the most most_common/ frequently used/polular words from the tweeet after clustering
            Args:
                terms: list of terms of clusters
            Returns:
                most top 5 frequently used words.
            We can also pass frequency as parameter 
        '''
    count_all = Counter()
    count_all.update(terms)
    return count_all.most_common(5)

def plot_data(q):
    '''
            Method to plot the data / cluster points after applying clustering on test data.
            This is the target method of python mulitprocessing 
            Args:
                q: A queue which contains the coordinates for plotting.
            Returns:
        '''
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    from pylab import rcParams
    import numpy as np

    plt.ioff() 
    llon = -130
    ulon = 100
    llat = -30
    ulat = 60
    rcParams['figure.figsize'] = (14,10)
    my_map = Basemap(projection='merc',
                resolution = 'l', area_thresh = 1000.0,
                llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
                urcrnrlon=ulon, urcrnrlat=ulat) #max longitude (urcrnrlon) and latitude (urcrnrlat)

    my_map.drawcoastlines()
    my_map.drawcountries()
    my_map.drawmapboundary()
    #my_map.fillcontinents(color = 'white', alpha = 0.3)
    #my_map.shadedrelief()
    #plt.pause(0.0001)
    #plt.show()


    colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, clusterNum))

    while True:
        if q.empty():
            time.sleep(5)

        else:
            # when queue is not empty plot the results
            obj=q.get()
            d=[x[0][0] for x in obj]
            c=[x[1] for x in obj]
            data = np.array(d)
            pcolor=np.array(c)
            try:
                xs,ys = my_map(data[:, 0], data[:, 1])
                my_map.scatter(xs, ys,  marker='o', alpha = 0.5,color=colors[pcolor])
                plt.pause(0.0001)
                plt.draw()
                plt.savefig("Clusters_plot_on_world_map.png")
                time.sleep(5)
            except IndexError: # Empty array
                pass


if __name__ == "__main__":
    '''
           Main method which does following
           1. Reads data from kafka topic - 'twitter-topic' in a BATCH_INTERVAL
           2. Reads the preivously build word2vec model using parquet
           3. Prepocesses(tokenize) feed data
           4. Prepares test and trainingData
           5. Runs streaming k-means model
           6. Clustering is based on geo location (using lat, long) and feeds text simliarity using word2vec.
           7. Plots the identified clusters and data using matplotlib. To achiece this I have used mulitprocessing with Queues as its a streaming system
           TODO: Persist the model to HDFS/ local files system for later use.
        '''
    # Multiprocessing
    q = multiprocessing.Queue()
    job_for_plotting = multiprocessing.Process(target=plot_data,args=(q,))
    job_for_plotting.daemon=True
    job_for_plotting.start()

    # Intialized Spark config
    #conf = SparkConf().setAppName("Kafka-Spark-Twitter-Feed-Clustering")
    #sc = SparkContext(conf=conf)
    sc = SparkContext.getOrCreate()
    sc.setLogLevel("ERROR")
    # get streaming context of batch interval 10 seconds
    #  TODO:
    #  Change this based on number of kakfka partitions , time to process current batch.
    # The above point is very crucial in prouction systems to achieve better paralelism in spark and handle backpresure
    stream=StreamingContext(sc,10) #
    
    # Read the stream into dstreams
    # Note : this is the loclahost mode
    Streaming = stream.socketTextStream("localhost", 9999)
    #print(kafkaStream.pprint())
    # Read word2vector model built offline using parquet
    sqlContext=SQLContext(sc)
    word2vec_model = sqlContext.read.parquet("resources/word2vector_model/data").alias("word2vec_model")
    #print(lookup.rdd.collectAsMap())
    print("Read Word2Vec model from parquet")
    # Boradcasting is used to avoid the copy of model in every machine/worker nodes
    word2vec_model_brooadcast = sc.broadcast(word2vec_model.rdd.collectAsMap())
    # The below code does folowing
    # Converts kafka event to json
    # Filters them based on created_at field in json. 
    # TODO: We can use Avro for schema management
    #Gets Coordinates lat,lng, and tokenize tweet text and append word2vect vector for every token
    print("Processing tweets....")
    processed_tweets= Streaming.map(lambda tweet : get_json_from_string(tweet)).filter(lambda tweet: tweet != False).filter(lambda tweet: 'created_at' in tweet).map(lambda tweet: (get_cordinates_from_event(tweet["text"])[0],get_cordinates_from_event(tweet["text"])[1],tweet["text"],media_url(tweet)))
    print("Processed tweets")
    processed_tweets.pprint()


Read Word2Vec model from parquet
Processing tweets....
Processed tweets


In [2]:
stream.start()
stream.awaitTermination()

-------------------------------------------
Time: 2021-05-03 02:36:40
-------------------------------------------
(0, 0, '* walks * \n* causes an earthquake *', False)
(0, 0, 'RT @eqgr: 5⃣0⃣ 🟠 Earthquake Mw 5.0, 02/05 20:49 (UTC), depth: 10 Km, NORTHERN SUMATRA, INDONESIA.\n\nSource: GFZ Potsdam -&gt; https://t.co/11WV…', False)
(0, 0, 'Unacceptable', False)
(0, 0, 'RT @IMPACTors_info: 🥚5/2UP\nIMPACTors「新曲披露！」 \n\n#少年倶楽部\n#少クラ\n#IMPACTors\n#佐藤新\n#基俊介\n#鈴木大河\n#影山拓也\n#松井奏\n#横原悠毅\n#椿泰我\n#Wildfire\n\n出演：IMPACTors\nhttps://…', False)

-------------------------------------------
Time: 2021-05-03 02:36:50
-------------------------------------------
(0, 0, 'RT @HostWithTheM0st: The disasters:\n\nNatielle - Earthquake\nDerek - Tsunami\nAsher - Tornado\nChurro - Volcanic Eruption\nDrizzle - Snowstorm', False)
(0, 0, 'RT @Cryptotalees: Teknik analize nasıl başlanmalı ile ilgili daha önce flood yapmıştım çok aşağıda kaldığı için tekrardan paylaşayım;', False)
(0, 0, 'RT @bruce_haigh: #auspol Which

-------------------------------------------
Time: 2021-05-03 02:38:20
-------------------------------------------
(0, 0, "RT @lynnfromtx: 1/? supporting local mutual aid and direct action is even more crucial under prop b's reinstatement (criminalizing homeless…", False)
(0, 0, 'RT @Wild_Gramps: After missing a year in our long term research last year due to COVID (still frustrates us). We are back in the Gariwerd G…', False)
(0, 0, 'RT @bruce_haigh: #auspol Which are the people the Australian government will protect? Not Indigenous Australians, not the poor, unemployed…', False)
(0, 0, 'RT @KaliHoochie: Massaging the flood gates 💦💦 https://t.co/nMhno7Stnt', 'http://pbs.twimg.com/ext_tw_video_thumb/1333480699253444617/pu/img/elnOayo3NWy2XOTC.jpg')
(0, 0, 'RT @earthquakesSF: A 2.3 magnitude earthquake occurred 0.62mi W of Danville, CA. Details: https://t.co/gKywbzU77b Map: https://t.co/KAQ0L6H…', False)

-------------------------------------------
Time: 2021-05-03 02:38:30
-------------

-------------------------------------------
Time: 2021-05-03 02:40:10
-------------------------------------------
(0, 0, 'RT @russdiemon: DROPPIN A SONG A WEEK AND IGNORING EVERYBODY STARTING THIS FRIDAY\n\nFRIDAY MAY 7 THE FLOOD BEGINS\n\nthese pictures are only r…', False)
(0, 0, "RT @CawthornforNC: Biden's latest action on the border is to cancel $17 billion worth of border projects as 1,000 illegal immigrants flood…", False)
(0, 0, 'RT @EQAlerts: #Earthquake M4.8 SANTA CRUZ ISLANDS, Solomon Islands  2 May 09:15 UTC - report/info: \nhttps://t.co/uszVDUbjCn', False)
(0, 0, 'RT @WeatherNation: 👀Whoa! A 6.6 earthquake occurred just off the east coast of Japan on Saturday! You can see the street markers really rat…', False)
(0, 0, 'RT @dgoold: Nolan Arenado wanted out in the middle of his contract so that he could play at a place of his choosing.\n\nThe reigning MVP in t…', False)
(0, 0, '#Earthquake - #زلزله Mag: 5 Depth : 63.97 Km  Place :  82 km WSW of Banda Aceh  Indonesia Date/Time

Py4JJavaError: An error occurred while calling o22.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/asus/spark-3.0.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/home/asus/spark-3.0.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 173, in takeAndPrint
    taken = rdd.take(num + 1)
  File "/home/asus/spark-3.0.1-bin-hadoop3.2/python/pyspark/rdd.py", line 1446, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "/home/asus/spark-3.0.1-bin-hadoop3.2/python/pyspark/context.py", line 1118, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "/home/asus/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/home/asus/spark-3.0.1-bin-hadoop3.2/python/pyspark/sql/utils.py", line 128, in deco
  

-------------------------------------------
Time: 2021-05-03 02:41:00
-------------------------------------------
(0, 0, 'I knew there were a bunch of nuts in my age group. They are embarrassing. They are the nuts who somehow graduated h… https://t.co/jiXrMXJEVS', False)
(0, 0, '@Momo06573806 Flood my cash app $lilmamacc304', False)
(0, 0, 'RT @JRfromStrickley: Our herd of Dairy Shorthorns grazing Brant Strickley, turning steep pasture grass into a valuable organic drink called…', False)
(0, 0, '#Sismo M 5 NORTHERN SUMATRA, #INDONESIA. 02-05-2021 20:49 UTC https://t.co/k3t1IuQb5W #Temblor #CSEM #EMSC… https://t.co/zNdxfnzIYO', False)
(0, 0, 'RT @Voices4Seniors: A group of women, mostly over the age of 55, supporting Cuomo coordinate to flood his press conferences and impeachment…', False)
(0, 0, "RT @Reuters: ICYMI: The picture-postcard Croatian resort of Porec is making sure it's prepared for an expected flood of tourists from mid-M…", False)

-----------------------------------------

-------------------------------------------
Time: 2021-05-03 02:42:40
-------------------------------------------
(0, 0, "@ekremalpay8 @Besiktas Reis derbi flood'u ne zaman gelir tahminen ?", False)
(0, 0, 'RT @BharadwajSpeaks: Malatipur in Malda is another seat which has witnessed huge illegal immigration.\n\nThe Mμslim population in this assemb…', False)
(0, 0, "RT @nypost: 'Cuomosexuals' form Facebook fan group, flood state hotline https://t.co/lUddMhBlqi https://t.co/iJ3RaeZYq0", 'http://pbs.twimg.com/media/E0aVs8iWEAcaSCR.jpg')
(0, 0, 'RT @EQAlerts: #Earthquake M4.0 Ceram Sea, Indonesia  2 May 13:04 UTC - report/info: \nhttps://t.co/qaVgVTDKJ8', False)
(0, 0, 'RT @TheSmartRadio: #earthquake Magnitudo 5.0 - 82 km WSW of Banda Aceh, Indonesia (15 minutes 18 seconds ago)', False)
(0, 0, 'RT @Deleinko: Take out time to pray for missionaries and unreached people groups. May the gospel spread like wildfire!', False)
(0, 0, 'RT @ChurchOfPumps: #Dogeism is spreading like wildfire. 🔥\n\nht

-------------------------------------------
Time: 2021-05-03 02:44:10
-------------------------------------------
(0, 0, 'RT @5thsound: Second lab flood in 17 months. Bad facilities at @UAlberta .  It is raining the labs.  Both times on a Sunday also. https://t…', 'http://pbs.twimg.com/media/E0Z-3_YUUAIgQYK.jpg')
(0, 0, 'RT @chub_la: Found this gem in my phone back from my birthday that never got posted when @CubJny came to LA to pound me out and boy did he…', False)
(0, 0, 'RT @Deleinko: Take out time to pray for missionaries and unreached people groups. May the gospel spread like wildfire!', False)
(0, 0, 'Pretty telling that @RichAzzopardi thanks this group for calling the impeachment hotline investigating Andrew Cuomo… https://t.co/SmZGmw5pfE', False)
(0, 0, 'RT @FatalisT69: Strong M6.8 earthquake in Japan, ⚡ Miyagi quake\n📅 May 1.\n日本の強いM6.8地震、宮城\n\n #miyagi #honshu #japan #earthquake #seism #japane…', False)
(0, 0, 'Got the neighbors yelling earthquake. 4.5 when I make the bed sh

-------------------------------------------
Time: 2021-05-03 02:45:50
-------------------------------------------
(0, 0, 'RT @akpartiistanbul: İBB’nin sorumluluğundaki Adalar’ın atlarına NE OLDU?\n978 at NEREDE?\n\nUzun süredir akıbeti meçhul olan ve CHP’li İBB yö…', False)
(0, 0, 'RT @MonkeeyCS: AWP WILDFIRE GA! (30$) 💸\n\n🍌RETWEET &amp; FOLLOW!\n🍌LIKE &amp; COMMENT: https://t.co/ACzsvvzoOf\n\nItem can be changed for cash if wante…', False)
(0, 0, 'RT @akpartiistanbul: İBB’nin sorumluluğundaki Adalar’ın atlarına NE OLDU?\n978 at NEREDE?\n\nUzun süredir akıbeti meçhul olan ve CHP’li İBB yö…', False)
(0, 0, "RT @osmannnurika: İBB 'hayvan sevgisi' dersinden de kaldı.\n\nAdalar'da satın alınan 1200 atın 978 tanesi kayıp. Öyle reklam filmi çekip, afi…", False)
(0, 0, '@libramoonstone @HellHat80172147 Healing is like the Cyclone roller coaster at Coney Island', False)
(0, 0, 'RT @denisevasi: If shorts are on your “do not wear list,” you are not alone. This time of year, requests for  altern

-------------------------------------------
Time: 2021-05-03 02:47:30
-------------------------------------------
(0, 0, '【地震情報】 06:17 長野県 木祖 131、石川県 能都 92、愛知県 美浜 22、秋田県 宮田 7 協和 2、福島県 桧枝岐 3、北海道 釧路支庁 厚岸 #地震 #jishin #earthquake #saigai #速報値 https://t.co/cu1CoJzIqR', 'http://pbs.twimg.com/media/E0aZEcbUYAALA05.png')
(0, 0, "RT @CawthornforNC: Biden's latest action on the border is to cancel $17 billion worth of border projects as 1,000 illegal immigrants flood…", False)
(0, 0, 'RT @BearDunOfficial: I don’t have any new photos becuase @SteeleEroticArt hasn’t yelled at me to get naked recently. \n\nYou all should go fl…', False)
(0, 0, "RT @harunkzlb01: Başlasın İtikaf...\nRamazanın son on gününe kavuştuk. Efendimiz'in unutulmuş sünnetini, itikaf ibadetini ihya edecek olmanı…", False)

-------------------------------------------
Time: 2021-05-03 02:47:40
-------------------------------------------
(0, 0, "Subhanallah.\nI invested all my savings to starting a poultry just recently, they all 

-------------------------------------------
Time: 2021-05-03 02:49:30
-------------------------------------------
(0, 0, 'RT @egebasaran: sabah sabah şöyle bir flood’a denk geldim ve çok şefkatli buldum. katılıyor ve arttırıyorum: zamanı geldiğinde bazı kişiler…', False)
(0, 0, 'RT @NdubuakuUchenna: They intentionally creat the insecurity in the north inorder the refugees to flood in the south in the name of one nig…', False)



In [ ]:
(0, 0, 'RT @RebeccaH2030: The moon rising over a hill in California that is engulfed in a wildfire looks really eerie.\n#Tiredearth https://t.co/62v…', 'http://pbs.twimg.com/media/E0TDMzcXEAMIIMd.png')

In [2]:
b=b'{"created_at": "Sun May 02 20:17:17 +0000 2021", "id": 1388950744158638081, "id_str": "1388950744158638081", "text": "RT @iamPetu_: Four pictures from Wizkid looking so simple and soft. Add yours let\\u2019s flood the tl\\ud83e\\udd29\\ud83e\\udd85\\n\\n#BattleofWizkidPictures https://t.co/7O\\u2026", "source": "<a href=\\"http://twitter.com/download/iphone\\" rel=\\"nofollow\\">Twitter for iPhone</a>", "truncated": false, "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 1210485638094639104, "id_str": "1210485638094639104", "name": "Made in Lagos \\u2764\\ufe0f\\ud83e\\udd85", "screen_name": "Rahman_DC1", "location": "Starboy", "url": null, "description": "i just wanna see the sun shine tomorrow..I Stan The GOAT Wizkid\\u2764\\ufe0f\\u2b50\\ufe0f\\ud83e\\udd851X Grammy Winner...Ronaldo The GOAT \\ud83d\\udc10", "translator_type": "none", "protected": false, "verified": false, "followers_count": 2674, "friends_count": 4995, "listed_count": 1, "favourites_count": 212345, "statuses_count": 112993, "created_at": "Fri Dec 27 09:00:49 +0000 2019", "utc_offset": null, "time_zone": null, "geo_enabled": false, "lang": null, "contributors_enabled": false, "is_translator": false, "profile_background_color": "F5F8FA", "profile_background_image_url": "", "profile_background_image_url_https": "", "profile_background_tile": false, "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "profile_image_url": "http://pbs.twimg.com/profile_images/1381920617600008198/ONjK68QP_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1381920617600008198/ONjK68QP_normal.jpg", "profile_banner_url": "https://pbs.twimg.com/profile_banners/1210485638094639104/1605878863", "default_profile": true, "default_profile_image": false, "following": null, "follow_request_sent": null, "notifications": null, "withheld_in_countries": []}, "geo": null, "coordinates": null, "place": null, "contributors": null, "retweeted_status": {"created_at": "Sun May 02 19:14:08 +0000 2021", "id": 1388934853387767813, "id_str": "1388934853387767813", "text": "Four pictures from Wizkid looking so simple and soft. Add yours let\\u2019s flood the tl\\ud83e\\udd29\\ud83e\\udd85\\n\\n#BattleofWizkidPictures https://t.co/7OJ8ForJhv", "display_text_range": [0, 109], "source": "<a href=\\"http://twitter.com/download/iphone\\" rel=\\"nofollow\\">Twitter for iPhone</a>", "truncated": false, "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 2909904461, "id_str": "2909904461", "name": "Petu \\ud83d\\uddfd", "screen_name": "iamPetu_", "location": " Only God can Save", "url": "https://curiouscat.qa/iamPetu_", "description": "ease your mind....Check @krisayyor bio", "translator_type": "none", "protected": false, "verified": false, "followers_count": 8480, "friends_count": 3628, "listed_count": 59, "favourites_count": 259897, "statuses_count": 189271, "created_at": "Sun Dec 07 21:27:46 +0000 2014", "utc_offset": null, "time_zone": null, "geo_enabled": false, "lang": null, "contributors_enabled": false, "is_translator": false, "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_tile": false, "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "profile_image_url": "http://pbs.twimg.com/profile_images/1353109831524167686/1uyea4b5_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1353109831524167686/1uyea4b5_normal.jpg", "profile_banner_url": "https://pbs.twimg.com/profile_banners/2909904461/1610762327", "default_profile": true, "default_profile_image": false, "following": null, "follow_request_sent": null, "notifications": null, "withheld_in_countries": []}, "geo": null, "coordinates": null, "place": null, "contributors": null, "is_quote_status": false, "quote_count": 3, "reply_count": 13, "retweet_count": 68, "favorite_count": 141, "entities": {"hashtags": [{"text": "BattleofWizkidPictures", "indices": [86, 109]}], "urls": [], "user_mentions": [], "symbols": [], "media": [{"id": 1388934840133750787, "id_str": "1388934840133750787", "indices": [110, 133], "media_url": "http://pbs.twimg.com/media/E0Z84NJWEAMYU60.jpg", "media_url_https": "https://pbs.twimg.com/media/E0Z84NJWEAMYU60.jpg", "url": "https://t.co/7OJ8ForJhv", "display_url": "pic.twitter.com/7OJ8ForJhv", "expanded_url": "https://twitter.com/iamPetu_/status/1388934853387767813/photo/1", "type": "photo", "sizes": {"thumb": {"w": 150, "h": 150, "resize": "crop"}, "small": {"w": 680, "h": 588, "resize": "fit"}, "medium": {"w": 749, "h": 648, "resize": "fit"}, "large": {"w": 749, "h": 648, "resize": "fit"}}}]}, "extended_entities": {"media": [{"id": 1388934840133750787, "id_str": "1388934840133750787", "indices": [110, 133], "media_url": "http://pbs.twimg.com/media/E0Z84NJWEAMYU60.jpg", "media_url_https": "https://pbs.twimg.com/media/E0Z84NJWEAMYU60.jpg", "url": "https://t.co/7OJ8ForJhv", "display_url": "pic.twitter.com/7OJ8ForJhv", "expanded_url": "https://twitter.com/iamPetu_/status/1388934853387767813/photo/1", "type": "photo", "sizes": {"thumb": {"w": 150, "h": 150, "resize": "crop"}, "small": {"w": 680, "h": 588, "resize": "fit"}, "medium": {"w": 749, "h": 648, "resize": "fit"}, "large": {"w": 749, "h": 648, "resize": "fit"}}}, {"id": 1388934840158928898, "id_str": "1388934840158928898", "indices": [110, 133], "media_url": "http://pbs.twimg.com/media/E0Z84NPWQAIA3at.jpg", "media_url_https": "https://pbs.twimg.com/media/E0Z84NPWQAIA3at.jpg", "url": "https://t.co/7OJ8ForJhv", "display_url": "pic.twitter.com/7OJ8ForJhv", "expanded_url": "https://twitter.com/iamPetu_/status/1388934853387767813/photo/1", "type": "photo", "sizes": {"large": {"w": 1242, "h": 1399, "resize": "fit"}, "medium": {"w": 1065, "h": 1200, "resize": "fit"}, "thumb": {"w": 150, "h": 150, "resize": "crop"}, "small": {"w": 604, "h": 680, "resize": "fit"}}}, {"id": 1388934840146436100, "id_str": "1388934840146436100", "indices": [110, 133], "media_url": "http://pbs.twimg.com/media/E0Z84NMXoAQrWnu.jpg", "media_url_https": "https://pbs.twimg.com/media/E0Z84NMXoAQrWnu.jpg", "url": "https://t.co/7OJ8ForJhv", "display_url": "pic.twitter.com/7OJ8ForJhv", "expanded_url": "https://twitter.com/iamPetu_/status/1388934853387767813/photo/1", "type": "photo", "sizes": {"thumb": {"w": 150, "h": 150, "resize": "crop"}, "large": {"w": 426, "h": 681, "resize": "fit"}, "small": {"w": 425, "h": 680, "resize": "fit"}, "medium": {"w": 426, "h": 681, "resize": "fit"}}}, {"id": 1388934840150597639, "id_str": "1388934840150597639", "indices": [110, 133], "media_url": "http://pbs.twimg.com/media/E0Z84NNXIAc7o2I.jpg", "media_url_https": "https://pbs.twimg.com/media/E0Z84NNXIAc7o2I.jpg", "url": "https://t.co/7OJ8ForJhv", "display_url": "pic.twitter.com/7OJ8ForJhv", "expanded_url": "https://twitter.com/iamPetu_/status/1388934853387767813/photo/1", "type": "photo", "sizes": {"thumb": {"w": 150, "h": 150, "resize": "crop"}, "large": {"w": 720, "h": 679, "resize": "fit"}, "medium": {"w": 720, "h": 679, "resize": "fit"}, "small": {"w": 680, "h": 641, "resize": "fit"}}}]}, "favorited": false, "retweeted": false, "possibly_sensitive": false, "filter_level": "low", "lang": "en"}, "is_quote_status": false, "quote_count": 0, "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "entities": {"hashtags": [{"text": "BattleofWizkidPictures", "indices": [100, 123]}], "urls": [], "user_mentions": [{"screen_name": "iamPetu_", "name": "Petu \\ud83d\\uddfd", "id": 2909904461, "id_str": "2909904461", "indices": [3, 12]}], "symbols": []}, "favorited": false, "retweeted": false, "filter_level": "low", "lang": "en", "timestamp_ms": "1619986637521"}\n'

In [6]:
li

{'created_at': 'Sun May 02 20:17:17 +0000 2021',
 'id': 1388950744158638081,
 'id_str': '1388950744158638081',
 'text': 'RT @iamPetu_: Four pictures from Wizkid looking so simple and soft. Add yours let’s flood the tl🤩🦅\n\n#BattleofWizkidPictures https://t.co/7O…',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1210485638094639104,
  'id_str': '1210485638094639104',
  'name': 'Made in Lagos ❤️🦅',
  'screen_name': 'Rahman_DC1',
  'location': 'Starboy',
  'url': None,
  'description': 'i just wanna see the sun shine tomorrow..I Stan The GOAT Wizkid❤️⭐️🦅1X Grammy Winner...Ronaldo The GOAT 🐐',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 2674,
  'friends_count': 4995,
  'listed_count': 1,
  'fa

In [4]:
import json
li=json.loads(b)

In [4]:
def media_url(tweet):
    if "media" in li['entities']:
        for image in li['entities']['media']:
            return image['media_url']
    try:
        if "media" in li['retweeted_status']['entities']:
            for image in  li['retweeted_status']['entities']['media']:
                return image['media_url']
    except KeyError:
        return False
    else:
        return False

In [5]:
b2=b'{"created_at": "Sun May 02 20:17:57 +0000 2021", "id": 1388950909971886080, "id_str": "1388950909971886080", "text": "\\u3010\\u5730\\u9707\\u60c5\\u5831\\u3011 [\\u5730\\u4e2d\\u30ab\\u30a6\\u30f3\\u30c8\\u5897\\u52a0] 05:17 \\u9577\\u91ce\\u770c \\u6728\\u7956 33 #\\u5730\\u9707 #jishin #earthquake #saigai #\\u901f\\u5831\\u5024 https://t.co/7bAk7jaDVz", "display_text_range": [0, 70], "source": "<a href=\\"http://kyomoniex.web.fc2.com/\\" rel=\\"nofollow\\">Kyoshin Monitor Extension</a>", "truncated": false, "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 100881698, "id_str": "100881698", "name": "\\u305f\\u304b\\u2139\\ufe0e5/3GW\\u4f11\\ud83c\\udf8c\\u61b2\\u6cd5\\u8a18\\u5ff5\\u65e5\\ud83d\\udea85/11\\u307e\\u3067\\u7dca\\u6025\\u4e8b\\u614b\\u5ba3\\u8a00\\u4e2d\\ud83d\\udea8\\u533b\\u7642\\u975e\\u5e38\\u4e8b\\u614b\\u30fb\\u8513\\u5ef6\\u9632\\u6b62\\uff08\\u2600\\uff09", "screen_name": "twtaka_jp", "location": null, "url": "http://twilog.org/twtaka_jp", "description": "From:Ehime\\u25b7Now:Osaka\\u3002\\u871c\\u67d1\\u56fd\\u306e\\u6c11\\u3002\\u611b\\u5a9b\\u770c\\u7523\\u306b\\u306f\\u611b\\u304c\\u3042\\u308b\\u3002\\u30d1\\u30ef\\u30e2\\u30d0@\\u7fbd\\u6c11\\u3001\\u6f22\\u691c2\\u7d1a\\u3001\\u786c\\u7b463\\u7d1a\\u3001\\u66f8\\u9053\\u516b\\u6bb5\\u3002nana\\u6c11/\\u97f3\\u697d\\u597d\\u304d/\\u30cb\\u30b3\\u52d5\\uff65\\u751f\\uff65\\u5b9f\\u6cc1/\\u30c4\\u30a4\\u30ad\\u30e3\\u30b9/LINE/Abema/Instagram/YouTube/\\u30a2\\u30cb\\u30e1/\\u5730\\u9707\\u60c5\\u5831 http://instagram.com/instakagra.m/", "translator_type": "none", "protected": false, "verified": false, "followers_count": 1823, "friends_count": 1767, "listed_count": 50, "favourites_count": 77184, "statuses_count": 712299, "created_at": "Fri Jan 01 02:07:39 +0000 2010", "utc_offset": null, "time_zone": null, "geo_enabled": true, "lang": null, "contributors_enabled": false, "is_translator": false, "profile_background_color": "89C9FA", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme4/bg.gif", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme4/bg.gif", "profile_background_tile": false, "profile_link_color": "FAB81E", "profile_sidebar_border_color": "EEEEEE", "profile_sidebar_fill_color": "FFFFFF", "profile_text_color": "333333", "profile_use_background_image": true, "profile_image_url": "http://pbs.twimg.com/profile_images/1286678945031544833/wqn9dIZw_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1286678945031544833/wqn9dIZw_normal.jpg", "profile_banner_url": "https://pbs.twimg.com/profile_banners/100881698/1542475299", "default_profile": false, "default_profile_image": false, "following": null, "follow_request_sent": null, "notifications": null, "withheld_in_countries": []}, "geo": null, "coordinates": null, "place": null, "contributors": null, "is_quote_status": false, "quote_count": 0, "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "entities": {"hashtags": [{"text": "\\u5730\\u9707", "indices": [34, 37]}, {"text": "jishin", "indices": [38, 45]}, {"text": "earthquake", "indices": [46, 57]}, {"text": "saigai", "indices": [58, 65]}, {"text": "\\u901f\\u5831\\u5024", "indices": [66, 70]}], "urls": [], "user_mentions": [], "symbols": [], "media": [{"id": 1388950908982095872, "id_str": "1388950908982095872", "indices": [71, 94], "media_url": "http://pbs.twimg.com/media/E0aLfiRVcAAu7bD.png", "media_url_https": "https://pbs.twimg.com/media/E0aLfiRVcAAu7bD.png", "url": "https://t.co/7bAk7jaDVz", "display_url": "pic.twitter.com/7bAk7jaDVz", "expanded_url": "https://twitter.com/twtaka_jp/status/1388950909971886080/photo/1", "type": "photo", "sizes": {"medium": {"w": 512, "h": 300, "resize": "fit"}, "large": {"w": 512, "h": 300, "resize": "fit"}, "small": {"w": 512, "h": 300, "resize": "fit"}, "thumb": {"w": 150, "h": 150, "resize": "crop"}}}]}, "extended_entities": {"media": [{"id": 1388950908982095872, "id_str": "1388950908982095872", "indices": [71, 94], "media_url": "http://pbs.twimg.com/media/E0aLfiRVcAAu7bD.png", "media_url_https": "https://pbs.twimg.com/media/E0aLfiRVcAAu7bD.png", "url": "https://t.co/7bAk7jaDVz", "display_url": "pic.twitter.com/7bAk7jaDVz", "expanded_url": "https://twitter.com/twtaka_jp/status/1388950909971886080/photo/1", "type": "photo", "sizes": {"medium": {"w": 512, "h": 300, "resize": "fit"}, "large": {"w": 512, "h": 300, "resize": "fit"}, "small": {"w": 512, "h": 300, "resize": "fit"}, "thumb": {"w": 150, "h": 150, "resize": "crop"}}}]}, "favorited": false, "retweeted": false, "possibly_sensitive": false, "filter_level": "low", "lang": "ja", "timestamp_ms": "1619986677054"}\n'

In [6]:
import json
li=json.loads(b2)

In [7]:
link=media_url(li)
print(link)

http://pbs.twimg.com/media/E0aLfiRVcAAu7bD.png


In [41]:
li={'created_at': 'Fri Apr 16 18:57:13 +0000 2021', 'id': 1383132390827073537, 'id_str': '1383132390827073537', 'text': "@jeanangel58 I'm sure it's happened, but those rare instances in no way justify the mad carnage our flood of civilian firearms cause.", 'display_text_range': [13, 133], 'source': 'Twitter for Android', 'truncated': False, 'in_reply_to_status_id': 1382999392194547712, 'in_reply_to_status_id_str': '1382999392194547712', 'in_reply_to_user_id': 1071359969201659904, 'in_reply_to_user_id_str': '1071359969201659904', 'in_reply_to_screen_name': 'jeanangel58', 'user': {'id': 281196145, 'id_str': '281196145', 'name': 'Herr Dr. Prof. ₥Ɇ₦₮₳Ⱡ ₩ł₥₱ wants Trump jailed', 'screen_name': 'timrchurch', 'location': 'Personal Time Cone', 'url': 'https://directory.sph.umn.edu/bio/sph-a-z/timothy-church', 'description': "I f'king hate cancer./Sure, the arc of history bends toward justice...but it ain't no glide path./tap, tap, tap Is this thing on?", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2983, 'friends_count': 3545, 'listed_count': 21, 'favourites_count': 140055, 'statuses_count': 38643, 'created_at': 'Tue Apr 12 20:40:54 +0000 2011', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': True, 'profile_link_color': '0084B4', 'profile_sidebar_border_color': 'FFFFFF', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1220083865009754112/8fTsq4ES_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1220083865009754112/8fTsq4ES_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/281196145/1582835938', 'default_profile': False, 'default_profile_image': False, 'following': None, 'follow_request_sent': None, 'notifications': None, 'withheld_in_countries': []}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'entities': {'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'jeanangel58', 'name': 'Jeanangel58', 'id': 1071359969201659904, 'id_str': '1071359969201659904', 'indices': [0, 12]}], 'symbols': []}, 'favorited': False, 'retweeted': False, 'filter_level': 'low', 'lang': 'en', 'timestamp_ms': '1618599433987'}

In [113]:
import requests
import tensorflow.keras
from PIL import Image, ImageOps, UnidentifiedImageError
import numpy as np
import pandas as pd

CLASSES=["Cyclone","EarthQuake","Flood","WildFire"]

photo_dir=[]
photo_dir.clear()

def CNN_classifier(link):
    counter=0
    pic="disaster"+str(counter)+".png"
    counter=counter+1
    photoname="res/"+pic
    photo_dir.append(photoname)
    response = requests.get(link)
    file = open(photoname, "wb")
    file.write(response.content)
    file.close()
    np.set_printoptions(suppress=True)
    model = tensorflow.keras.models.load_model('keras_model.h5')
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    problem_files=[]
    Value_err=[]
    try:
        for img in photo_dir:
            try:
                image = Image.open(img).convert('RGB')
            except UnidentifiedImageError:
                problem_files.append(img)
    except OSError as e:
        return "OSError"

        
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    try:
        data[0] = normalized_image_array
        #run the inference
        prediction = model.predict(data)
        lis=list(prediction[0])
        ind=lis.index(max(lis))
        pred_class=CLASSES[ind]
        return pred_class
    except ValueError:
        return "ValueError"
    except TypeError:
        return "TypeError"

In [116]:
results=CNN_classifier(link)

In [117]:
results

'WildFire'

In [8]:
import requests
import tensorflow.keras
from PIL import Image, ImageOps, UnidentifiedImageError
import numpy as np
import pandas as pd

CLASSES=["Cyclone","EarthQuake","Flood","WildFire"]

photo_dir=[]
photo_dir.clear()

def media_url(tweet):
    if "media" in tweet['entities']:
        for image in tweet['entities']['media']:
            return image['media_url']
    try:
        if "media" in tweet['retweeted_status']['entities']:
            for image in  tweet['retweeted_status']['entities']['media']:
                return image['media_url']
    except KeyError:
        return False
    else:
        return False

def CNN_classifier(tweet):
    link=media_url(tweet)
    if link==False:
        return False
    else:
        counter=0
        pic="disaster"+str(counter)+".png"
        counter=counter+1
        photoname="res/"+pic
        photo_dir.append(photoname)
        response = requests.get(link)
        file = open(photoname, "wb")
        file.write(response.content)
        file.close()
        np.set_printoptions(suppress=True)
        model = tensorflow.keras.models.load_model('keras_model.h5')
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        problem_files=[]
        Value_err=[]
        try:
            for img in photo_dir:
                try:
                    image = Image.open(img).convert('RGB')
                except UnidentifiedImageError:
                    problem_files.append(img)
        except OSError as e:
            return False

        
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.ANTIALIAS)
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        try:
            data[0] = normalized_image_array
            #run the inference
            prediction = model.predict(data)
            lis=list(prediction[0])
            ind=lis.index(max(lis))
            pred_class=CLASSES[ind]
            return pred_class
        except ValueError:
            return False
        except TypeError:
            return False

In [9]:
CNN_classifier(li)

'WildFire'